In [1]:
import numpy as np
import tensorflow as tf

from scipy.stats import gmean
from sklearn.metrics import confusion_matrix
from src.data.synthetic_gen import DataLoader
from src.models.classification import NeuralNetClassifier

In [2]:
dl = DataLoader(problem="classification", dataset_size=1)
datasets, minmax = dl.load_cross_validation_datasets()

train_data = datasets["train"]
valid_data = datasets["valid"]

### Training best neural network

- Model: Rede Neural #11
- Parameters
    - hidden units: [128, 128]
    - activation: relu
    - learning rate: 0.001
    - epochs: 500
    - batch size: 32

In [3]:
# Train best neural network
# Rede Neural #11
learning_rate = 0.001
epochs = 500
batch_size = 32

params = {"hidden_units": [128, 128], "activation": "relu"}

models = []
for train, valid in zip(train_data, valid_data):

    train_features, train_labels = train["features"], train["targets"]
    valid_features, valid_labels = valid["features"], valid["targets"]

    features, labels = train_features.values, train_labels.values
    train_ds = tf.data.Dataset.from_tensor_slices((features, labels)).shuffle(10000).batch(batch_size)

    features, labels = valid_features.values, valid_labels.values
    valid_ds = tf.data.Dataset.from_tensor_slices((features, labels)).batch(batch_size)

    loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.CategoricalAccuracy()
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(),
        # patience 10% of epochs size
        tf.keras.callbacks.EarlyStopping(min_delta=0.0001, patience=50),
    ]
    model = NeuralNetClassifier(**params)
    model.compile(optimizer=optimizer, loss=loss_object, metrics=[accuracy])
    h = model.fit(
        train_ds,
        epochs=epochs,
        validation_data=valid_ds,
        callbacks=callbacks,
        verbose=1,  # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch
    )

    models.append(model)

    # folds.append({"fold": fold + 1, "model": model, "history": pd.DataFrame(h.history)})
    # val_results = {k: np.round(v[-1], decimals=4) for k, v in h.history.items() if "val_" in k}

Epoch 1/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - categorical_accuracy: 0.8147 - loss: 0.4866 - val_categorical_accuracy: 0.9525 - val_loss: 0.1396 - learning_rate: 0.0010
Epoch 2/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - categorical_accuracy: 0.9555 - loss: 0.1260 - val_categorical_accuracy: 0.9715 - val_loss: 0.0843 - learning_rate: 0.0010
Epoch 3/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - categorical_accuracy: 0.9718 - loss: 0.0856 - val_categorical_accuracy: 0.9691 - val_loss: 0.0702 - learning_rate: 0.0010
Epoch 4/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 678us/step - categorical_accuracy: 0.9732 - loss: 0.0752 - val_categorical_accuracy: 0.9762 - val_loss: 0.0653 - learning_rate: 0.0010
Epoch 5/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 666us/step - categorical_accuracy: 0.9754 - loss: 0.0629 - val_categorical_accuracy: 0.9762 - val_loss: 0.0590 - learning_rate: 0.0010
Epoch 6/500
333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - categorical_accuracy: 0.9792 - loss: 0.0568 - val_

### Evaluation

In [14]:
confusion_matrix_values = []
accuracies = []
sensitivities = []
sp_indexes = []

for valid, model in zip(valid_data, models):
    valid_features, valid_labels = valid["features"], valid["targets"]

    X_valid = tf.convert_to_tensor(valid_features)
    probs = tf.convert_to_tensor(valid_labels)
    y_valid = tf.argmax(probs, axis=1)

    logits = model(X_valid)
    probs_hat = tf.nn.softmax(logits)
    y_valid_hat = tf.argmax(probs_hat, axis=1)

    # Confusion matrix
    cm = confusion_matrix(y_valid, y_valid_hat)
    confusion_matrix_values.append(cm)

    # Accuracy
    accuracies.append(np.diag(cm).sum() / cm.sum())

    # Sensitivity
    sens = np.diag(cm) / cm.sum(axis=1)
    sensitivities.append(sens)

    # SP Index
    sp_indexes.append(np.sqrt(np.mean(sens) * gmean(sens)))

confusion_matrix_values = np.array(confusion_matrix_values)
accuracies = np.array(accuracies) * 100
sensitivities = np.array(sensitivities) * 100
sp_indexes = np.array(sp_indexes) * 100

In [16]:
# print(confusion_matrix_values)
print(f"Accuracy = {accuracies.mean():.2f} +/- {accuracies.std() / np.sqrt(10 - 1):.2f} %")

mean_sens = sensitivities.mean(axis=0)
std_err_sens = sensitivities.std(axis=0) / np.sqrt(10 - 1)
print(f"Sensitivity [V] = {mean_sens[0]:.2f} +/- {std_err_sens[0]:.2f} %")
print(f"Sensitivity [VL] = {mean_sens[1]:.2f} +/- {std_err_sens[1]:.2f} %")
print(f"Sensitivity [L] = {mean_sens[2]:.2f} +/- {std_err_sens[2]:.2f} %")

print(f"SP Index = {sp_indexes.mean():.2f} +/- {sp_indexes.std() / np.sqrt(10 - 1):.2f} %")

Accuracy = 99.27 +/- 0.08 %
Sensitivity [V] = 99.04 +/- 0.22 %
Sensitivity [VL] = 99.18 +/- 0.17 %
Sensitivity [L] = 99.59 +/- 0.11 %
SP Index = 99.27 +/- 0.08 %
